<a href="https://colab.research.google.com/github/aletscn/NLP-REAL-OR-NOT-/blob/master/cleanTweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import os
import sys

import pandas as pd
import numpy as np
from collections import defaultdict
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop=set(stopwords.words('english'))
from nltk.tokenize import word_tokenize
import string
import re
from PIL import Image    # to import the image


%matplotlib inline

#para suprimir notacion cientifica en los outputs
pd.options.display.float_format='{:20,.1f}'.format

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#Donde se encuentran los documentos (train, text)
DRIVE_DIR='/content/drive'
DATA_DIR = 'drive/My Drive/Colab Notebooks'
TWEETS_DIR='data_tp2'
#GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')

In [5]:
from google.colab import drive
drive.mount(DRIVE_DIR)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
os.chdir(DATA_DIR)

In [7]:
ls

 20_newsgroup/                                  glove.twitter.27B.50d.txt
'22 5.ipynb'                                    glove.twitter.27B.zip
 34.submit.csv                                  glove.twitter.27B.zip.1
 cleanTweets.ipynb                              glove.twitter.27B.zip.2
'CNN Glove Embeddings.ipynb'                    glove.twitter.27B.zip.3
'CNN Glove twitter Embeddings.ipynb'            glove.twitter.27B.zip.4
'CNN on subwords.ipynb'                         glove.twitter.27B.zip.5
 CNN_tweets.ipynb                               log.csv
'CNN word2vec.ipynb'                            lstm2.csv
'Copia de CNN Glove twitter Embeddings.ipynb'   news20.tar.gz
'Copia de CNN on subwords.ipynb'                preprocessed_dataset.npz
'Copy of CNN_tweets (1).ipynb'                  pre-trained-model/
'Copy of CNN_tweets.ipynb'                      result.txt
'Copy of Ejercicio Clase 20-4-2020.ipynb'       sample_submission.xlsx
 data_helpers                                   su

In [148]:
#Loading CSVs

train=pd.read_csv(os.path.join(TWEETS_DIR, 'train.csv'))
test=pd.read_csv(os.path.join(TWEETS_DIR, 'test.csv'))
train=train.sample(frac=1).reset_index(drop=True)
train['keyword']=np.NaN
train['location']=np.NaN
#Filling NAs
train['keyword']=train['keyword'].fillna('no_keyword')
train['location']=train['location'].fillna('no_location')

test['keyword']=test['keyword'].fillna('no_keyword')
test['location']=test['location'].fillna('no_location')

#Filtering duplicated tweets, deleting contradictory labeled tweets (ONLY FOR TRAIN DF)

train_clean=train
duplicate_texts=train_clean.loc[:,['text','target']].groupby(['text']).transform('count')
train_clean['duplicate_text_count']=duplicate_texts

contradiction=train_clean.loc[:,['text','target']].groupby(['text']).transform('nunique')
train_clean['count_targets']=contradiction
train_clean=train_clean[train.count_targets==1]
train_clean.drop('count_targets',axis=1,inplace=True)

train_clean.drop_duplicates(subset ='text', keep = 'first', inplace = True)
print('El DataFrame post cleaning cuenta con {} filas y {} columnas'.format(train_clean.shape[0],train_clean.shape[1]))


El DataFrame post cleaning cuenta con 7485 filas y 6 columnas


,text,target,keyword,location,duplicate_text_count
0,03/08/11: Police stop a 41-year-old in the Bro...,1,no_keyword,no_location,1
1,The following is a list of the top 10 most pop...,0,no_keyword,no_location,1
2,I'm just so sorry for the pain and suffering t...,0,no_keyword,no_location,1
3,"— John M. Z. (@JohnMZZ) October 10, 2014 The ...",1,no_keyword,no_location,1
4,twitter.com/2FcK4XlF6Z — CNN Politics (@CNNPol...,1,no_keyword,no_location,1
...,...,...,...,...,...
11547,Don't let a few assholes ruin your night,0,no_keyword,no_location,1
11548,The people are going to give him a big speech....,0,no_keyword,no_location,1
11549,twitter.com/9EJh7JqBn — RT (@RT_com) October 1...,1,no_keyword,no_location,1
11550,#ClimateChange Eyewitness to Extreme Weather: ...,1,no_keyword,no_location,1


In [149]:
#Cleaning URLs, punctuations, special characters


def deleteSpecials(inputString):
  text = re.sub(r'http\S+', '', inputString)
  text_bis = re.sub(r'\S+.co\S+', '', text)
  text_bis2 = re.sub(r'\S+.org\S+', '', text_bis)
  text2 = re.sub(r'@\S+', '', text_bis2)
  text3 = re.sub(r'&\S+', '', text2)
  text4 = re.sub(r'Ã\S+', '', text3)
  text5 = re.sub(r'www\S+','',text4)
  text6 = re.sub(r'\x89Û\S+','',text4)
  return (text6)

def decontracted(inputString):
    phrase = re.sub(r"won't", "will not", inputString)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"couldn", "could", phrase)
    phrase = re.sub(r"don", "do", phrase)
    phrase = re.sub(r"doesn", "does", phrase)
    phrase = re.sub(r"isn", "is", phrase)
    phrase = re.sub(r"mustn", "must", phrase)
    phrase = re.sub(r"shouldn", "should", phrase)
    phrase = re.sub(r"wasn", "was", phrase)

    # general

    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    
    return phrase

def deletePunctuation(inputString):
  aux=string.punctuation
  special_characters='ÃŒ©‰¥¼ªû—'
  #puncts=aux.translate(str.maketrans("#@:/.","     "))
  auxstring=" " * len(aux)
  auxstring2=" " * len(special_characters)
  outputString=inputString.translate(str.maketrans(aux,auxstring))
  outputString=outputString.translate(str.maketrans(special_characters,auxstring2))
  return outputString

def deleteDigits(value):
    blist2 = [item for item in value if not item.isdigit()]
    blist3 = "".join(blist2)
    return blist3

def cleanText(inputString):
  text_1=deleteSpecials(inputString)
  text_2=decontracted(text_1)
  text_3=deletePunctuation(text_2)
  text_4=deleteDigits(text_3)
  return text_4



train_clean['text_clean'] = train_clean['text'].apply(lambda x: cleanText(x))
test_clean=test
test_clean['text_clean'] = test_clean['text'].apply(lambda x: cleanText(x))



In [150]:
train_clean['duplicate_text_clean_count']=train_clean.loc[:,['text_clean','target']].groupby(['text_clean']).transform('count')
train_clean['count_targets']=train_clean.loc[:,['text_clean','target']].groupby(['text_clean']).transform('nunique')
train_clean=train_clean[train_clean.count_targets==1]
train_clean.drop('count_targets',axis=1,inplace=True)
train_clean.drop_duplicates(subset ='text_clean', keep = 'first', inplace = True)
print('El DataFrame pre cleaning cuenta con {} filas y {} columnas'.format(train.shape[0],train.shape[1]))
print('El DataFrame post cleaning cuenta con {} filas y {} columnas'.format(train_clean.shape[0],train_clean.shape[1]))

train_clean.drop('duplicate_text_count',axis=1,inplace=True)
train_clean.drop('duplicate_text_clean_count',axis=1,inplace=True)


El DataFrame pre cleaning cuenta con 7613 filas y 7 columnas
El DataFrame post cleaning cuenta con 6855 filas y 8 columnas


In [151]:

nltk.download('punkt')
def deleteStopwords(inputString):
  inputString= inputString.lower()
  text_tokens = word_tokenize(inputString)
  tokens_without_sw = [word for word in text_tokens if not word in stopwords.words('english')]
  filtered_sentence = (" ").join(tokens_without_sw)
  return filtered_sentence

train_clean['text_clean_nosw'] = train_clean['text_clean'].apply(lambda x:deleteStopwords(x))
test_clean['text_clean_nosw'] = test_clean['text_clean'].apply(lambda x:deleteStopwords(x))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [152]:
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

def sentenceLem(sentence):
    wordnet_lemmatizer = WordNetLemmatizer()
    sentence_words = nltk.word_tokenize(sentence)
    temp_line=[] 
    lines_with_lemmas=[]
    
    for word in sentence_words:
        temp_line.append(wordnet_lemmatizer.lemmatize(word, pos="v"))

    string=' ' 
    lines_with_lemmas.append(string.join(temp_line)) 
    lines=lines_with_lemmas
    str1=''
    lines=str1.join(lines)
    return lines

train_clean['text_clean_lem'] = train_clean['text_clean_nosw'].apply(lambda x:sentenceLem(x))
test_clean['text_clean_lem'] = test_clean['text_clean_nosw'].apply(lambda x:sentenceLem(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [153]:
from nltk.tokenize import word_tokenize

tokenized_sents = [word_tokenize(i) for i in train_clean['text_clean_lem']]
train_clean['lem_tokenized']=tokenized_sents

tokenized_sents = [word_tokenize(i) for i in test_clean['text_clean_lem']]
test_clean['lem_tokenized']=tokenized_sents

In [154]:
def hasNumbers(inputString):
  return any(w.isdigit() for w in str(inputString).split())

def hasUrl(inputString):
  if '.co' in inputString or 'http' in inputString :
    return True
  else:
    return False

#train_no_duplicates['countries'] = train_no_duplicates['text_clean'].apply(lambda x: hasCountries

train_clean['url_count'] = train_clean['text'].apply(lambda x: hasUrl(str(x)))
train_clean['hashtag_count'] = train_clean['text'].apply(lambda x: any([c for c in str(x) if c == '#']))
train_clean['mention_count'] = train_clean['text'].apply(lambda x: any([c for c in str(x) if c == '@']))
train_clean['digits'] = train_clean['text'].apply(lambda x: hasNumbers(x))
train_clean['characters_count'] = train_clean['text'].apply(lambda x: len(str(x)))
train_clean['characters_count_clean'] = train_clean['text_clean'].apply(lambda x: len(str(x)))
train_clean['characters_count_clean_sw'] = train_clean['text_clean_nosw'].apply(lambda x: len(str(x)))
train_clean['characters_reduction'] = (train_clean['characters_count_clean']-train_clean['characters_count_clean_sw'])/train_clean['characters_count_clean']
train_clean['word_count'] = train_clean['text'].apply(lambda x: len(str(x).split()))
train_clean['word_count_clean'] = train_clean['text_clean'].apply(lambda x: len(str(x).split()))
train_clean['word_count_clean_sw'] = train_clean['text_clean_nosw'].apply(lambda x: len(str(x).split()))
train_clean['word_reduction'] = (train_clean['word_count_clean']-train_clean['word_count_clean_sw'])/train_clean['word_count_clean']
#train_no_duplicates['countries'] = train_no_duplicates['text_clean'].apply(lambda x: hasCountries

test_clean['url_count'] = test_clean['text'].apply(lambda x: hasUrl(str(x)))
test_clean['hashtag_count'] = test_clean['text'].apply(lambda x: any([c for c in str(x) if c == '#']))
test_clean['mention_count'] = test_clean['text'].apply(lambda x: any([c for c in str(x) if c == '@']))
test_clean['digits'] = test_clean['text_clean'].apply(lambda x: hasNumbers(x))
test_clean['characters_count'] = test_clean['text'].apply(lambda x: len(str(x)))
test_clean['characters_count_clean'] = test_clean['text_clean'].apply(lambda x: len(str(x)))
test_clean['characters_count_clean_sw'] = test_clean['text_clean_nosw'].apply(lambda x: len(str(x)))
test_clean['characters_reduction'] = (test_clean['characters_count_clean']-test_clean['characters_count_clean_sw'])/test_clean['characters_count_clean']
test_clean['word_count'] = test_clean['text'].apply(lambda x: len(str(x).split()))
test_clean['word_count_clean'] = test_clean['text_clean'].apply(lambda x: len(str(x).split()))
test_clean['word_count_clean_sw'] = test_clean['text_clean_nosw'].apply(lambda x: len(str(x).split()))
test_clean['word_reduction'] = (test_clean['word_count_clean']-test_clean['word_count_clean_sw'])/test_clean['word_count_clean']
#train_no_duplicates['countries'] = train_no_duplicates['text_clean'].apply(lambda x: hasCountries

,text,target,keyword,location,text_clean,text_clean_nosw,text_clean_lem,lem_tokenized,url_count,hashtag_count,mention_count,digits,characters_count,characters_count_clean,characters_count_clean_sw,characters_reduction,word_count,word_count_clean,word_count_clean_sw,word_reduction
11499,twitter.com/vCKZW2Z9qX — New York Times (@nyti...,0,no_keyword,no_location,New York Times November,new york times november,new york time november,"[new, york, time, november]",True,False,True,True,62,29,23,0.2,8,4,4,0.0
11500,Rolling sandunes the gentle lapping of the sea...,1,no_keyword,no_location,Rolling sandunes the gentle lapping of the sea...,rolling sandunes gentle lapping sea call gulls...,roll sandunes gentle lap sea call gull nuclear...,"[roll, sandunes, gentle, lap, sea, call, gull,...",True,True,False,False,119,97,71,0.3,18,17,10,0.4
11501,"So, they are saying that the word 'dmac1043' i...",0,no_keyword,no_location,So they are saying that the word wouldmac i...,saying word wouldmac areddish word areddish,say word wouldmac areddish word areddish,"[say, word, wouldmac, areddish, word, areddish]",False,False,False,False,91,93,43,0.5,17,17,6,0.6
11502,I also loved 'Bury my heart at wounded knee' t...,0,no_keyword,no_location,I also loved Bury my heart at wounded knee t...,also loved bury heart wounded knee thegame,also love bury heart wound knee thegame,"[also, love, bury, heart, wound, knee, thegame]",False,True,False,False,58,58,42,0.3,11,11,7,0.4
11503,Medieval airplane hijacker testa: earnings the...,1,no_keyword,no_location,Medieval airplane hijacker testa earnings the...,medieval airplane hijacker testa earnings dist...,medieval airplane hijacker testa earn distinct...,"[medieval, airplane, hijacker, testa, earn, di...",False,False,False,False,76,76,70,0.1,9,9,8,0.1
11504,I wish that the earth sea and sky up above\nwo...,0,no_keyword,no_location,I wish that the earth sea and sky up above\nwo...,wish earth sea sky would send someone lava,wish earth sea sky would send someone lava,"[wish, earth, sea, sky, would, send, someone, ...",False,False,False,False,76,76,42,0.4,16,16,8,0.5
11505,there's this person &amp; they reckon when you...,0,no_keyword,no_location,there is this person they reckon when you are...,person reckon dying brain floods dmt causing r...,person reckon die brain flood dmt cause relive...,"[person, reckon, die, brain, flood, dmt, cause...",False,False,False,False,143,140,77,0.5,26,27,12,0.6
11506,twitter.com/4BmJm6X6yU — The Atlantic (@theatl...,1,no_keyword,no_location,The Atlantic December,atlantic december,atlantic december,"[atlantic, december]",True,False,True,True,70,29,17,0.4,8,3,2,0.3
11507,#np Avenged Sevenfold - Hail To The King,0,no_keyword,no_location,np Avenged Sevenfold Hail To The King,np avenged sevenfold hail king,np avenge sevenfold hail king,"[np, avenge, sevenfold, hail, king]",False,True,False,False,40,40,30,0.2,8,7,5,0.3
11508,I'm not saying this is an easy series. It's no...,0,no_keyword,no_location,I am not saying this is an easy series It is ...,saying easy series easy series easy series saying,say easy series easy series easy series say,"[say, easy, series, easy, series, easy, series...",False,False,False,False,111,115,49,0.6,23,27,8,0.7


In [155]:
nan_rows = train_clean[train_clean['word_reduction'].isnull()]


In [156]:
nan_rows

,id,keyword,location,text,target,text_clean,text_clean_nosw,text_clean_lem,lem_tokenized,url_count,hashtag_count,mention_count,digits,characters_count,characters_count_clean,characters_count_clean_sw,characters_reduction,word_count,word_count_clean,word_count_clean_sw,word_reduction


In [144]:
train_clean.dropna(subset=['word_reduction'],inplace=True)


In [147]:
train_clean.to_csv("train_clean_aug.csv", index=False, header=True)
from google.colab import files
files.download('train_clean_aug.csv')
test_clean.to_csv("test_clean.csv", index=False, header=True)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ls

 20_newsgroup/                                  glove.twitter.27B.zip
'22 5.ipynb'                                    glove.twitter.27B.zip.1
 34.submit.csv                                  glove.twitter.27B.zip.2
 cleanTweets.ipynb                              glove.twitter.27B.zip.3
'CNN Glove Embeddings.ipynb'                    glove.twitter.27B.zip.4
'CNN Glove twitter Embeddings.ipynb'            glove.twitter.27B.zip.5
'CNN on subwords.ipynb'                         log.csv
 CNN_tweets.ipynb                               news20.tar.gz
'CNN word2vec.ipynb'                            preprocessed_dataset.npz
'Copia de CNN Glove twitter Embeddings.ipynb'   pre-trained-model/
'Copia de CNN on subwords.ipynb'                result.txt
'Copy of CNN_tweets (1).ipynb'                  sample_submission.xlsx
'Copy of CNN_tweets.ipynb'                      submission.csv
'Copy of Ejercicio Clase 20-4-2020.ipynb'       test_clean.csv
 data_helpers                                   tfidf
 d